In [1]:
!pip install "protobuf==3.20.3" --quiet
print('installed')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [2]:
import os

# <-- EDIT THIS: set to your dataset folder (contains class subfolders)
DATA_DIR = "/kaggle/input/cevi-img-dataset/Final Dataset - Cleaned (1)"

# Do not change below
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Dataset folder not found: {DATA_DIR}. Check the path.")

# Count images per class
classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
print("Found classes:", classes)
total = 0
for c in classes:
    cnt = len([f for f in os.listdir(os.path.join(DATA_DIR,c)) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    print(f"  {c}: {cnt} images")
    total += cnt
print("Total images:", total)

Found classes: ['gobi manchurian', 'momos', 'dhokla', 'capsicum', 'fried chicken', 'gujiya', 'orange', 'kofta', 'pav bhaji', 'tomato', 'kaju katli', 'turnip', 'rajma', 'missi roti', 'ginger', 'mysore pak', 'paratha', 'achar', 'lassi', 'papad', 'spring rolls', 'fried rice', 'kathi roll', 'aloo gobi', 'naan bread', 'pineapple', 'masala dosa', 'sandwich', 'aloo tikki', 'bagels', 'apple', 'palak paneer', 'bhindi masala', 'medu vada', 'besan cheela', 'lettuce', 'omelette', 'onion pakoda', 'beetroot', 'chow mein', 'kiwi', 'rasgulla', 'shankarpali', 'imarti', 'mishti doi', 'chicken rezala', 'pear', 'scrambled eggs', 'vada pav', 'ras malai', 'popcorn', 'toast', 'cabbage', 'kadai paneer', 'canned potatoes', 'chai', 'cooked pasta', 'basundi', 'chapati', 'cauliflower', 'phirni', 'sheera', 'sunny side up eggs', 'khandvi', 'kebabs', 'baingan bharta', 'malpua', 'shrikhand', 'idli', 'jalebi', 'appam', 'taco', 'kachori', 'litti chokha', 'stuffed karela', 'sheer khurma', 'kadhi pakoda', 'butter chicken

In [ ]:
# Full training pipeline (MobileNetV3-Large classifier)
# Paste this into Colab and run (use GPU runtime).

import os, math, numpy as np, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large, mobilenet_v3
from tensorflow.keras import layers, Model
from sklearn.metrics import confusion_matrix, classification_report
import itertools

# ---------- EDIT THESE ----------
DATA_DIR = "/kaggle/input/cevi-img-dataset/Final Dataset - Cleaned (1)"   # <- set this to your dataset root (class subfolders)
OUTPUT_DIR = "/kaggle/working/"
BATCH_SIZE = 32
IMG_SIZE = (224,224)
EPOCHS_HEAD = 12         # train head for this many epochs
EPOCHS_FINE = 6          # fine-tune epochs (after unfreeze)
VAL_SPLIT = 0.2
UNFREEZE_AT = -30        # unfreeze last ~30 layers of backbone for fine-tuning
# --------------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Using DATA_DIR:", DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)

# ----- Generators (correct preprocessing; DO NOT use rescale=1./255 when using preprocess_input) -----
train_gen = ImageDataGenerator(
    preprocessing_function=mobilenet_v3.preprocess_input,
    rotation_range=15, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.05, zoom_range=0.1, horizontal_flip=True,
    validation_split=VAL_SPLIT
)
val_gen = ImageDataGenerator(preprocessing_function=mobilenet_v3.preprocess_input, validation_split=VAL_SPLIT)

train_flow = train_gen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical',
    subset='training', seed=42
)
val_flow = val_gen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical',
    subset='validation', seed=42, shuffle=False
)

num_classes = train_flow.num_classes
print("Num classes:", num_classes)
print("Train samples:", train_flow.samples, "Val samples:", val_flow.samples)

# ----- Build model -----
backbone = MobileNetV3Large(input_shape=(IMG_SIZE[0],IMG_SIZE[1],3), include_top=False, weights='imagenet')
x = backbone.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=backbone.input, outputs=outputs)

# Freeze backbone initially
backbone.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----- Callbacks -----
best_path = os.path.join(OUTPUT_DIR, "best_mobilenetv3_large.h5")
cbs = [
    tf.keras.callbacks.ModelCheckpoint(best_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=False, verbose=1)
]

# ----- Train head -----
history = model.fit(
    train_flow,
    epochs=EPOCHS_HEAD,
    validation_data=val_flow,
    callbacks=cbs
)

# ----- Unfreeze & fine-tune -----
# Unfreeze last N layers
if UNFREEZE_AT < 0:
    for layer in backbone.layers[UNFREEZE_AT:]:
        layer.trainable = True
else:
    for layer in backbone.layers[UNFREEZE_AT:]:
        layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

history_ft = model.fit(
    train_flow,
    epochs=EPOCHS_FINE,
    validation_data=val_flow,
    callbacks=cbs
)

# ----- Load best model if present -----
if os.path.exists(best_path):
    model.load_weights(best_path)
    print("Loaded best checkpoint:", best_path)

# ----- Evaluate -----
val_steps = math.ceil(val_flow.samples / BATCH_SIZE)
print("Evaluating on validation set...")
res = model.evaluate(val_flow, steps=val_steps, verbose=1)
print("Val loss, val acc:", res)

# ----- Predict & Confusion Matrix -----
y_pred_probs = model.predict(val_flow, steps=val_steps, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = val_flow.classes
labels = list(val_flow.class_indices.keys())

print("Classification report:")
print(classification_report(y_true, y_pred, target_names=labels, zero_division=0))

cm = confusion_matrix(y_true, y_pred)
# Simple confusion plot (may be large for many classes)
plt.figure(figsize=(10,10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion matrix (validation)")
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

# ----- Save final model & convert to TFLite -----
final_saved = os.path.join(OUTPUT_DIR, "mobilenetv3_large_final.h5")
model.save(final_saved)
print("Saved final model:", final_saved)

try:
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_path = os.path.join(OUTPUT_DIR, "mobilenetv3_large.tflite")
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print("Saved TFLite:", tflite_path)
except Exception as e:
    print("TFLite conversion failed:", e)

# ----- Done -----
print("Training complete. Models & artifacts are in:", OUTPUT_DIR)


2025-11-30 05:24:08.821936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764480249.022767      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764480249.074061      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using DATA_DIR: /kaggle/input/cevi-img-dataset/Final Dataset - Cleaned (1)
OUTPUT_DIR: /kaggle/working/
Found 14051 images belonging to 124 classes.
Found 3449 images belonging to 124 classes.
Num classes: 124
Train samples: 14051 Val samples: 3449


I0000 00:00:1764480268.556665      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764480268.557302      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

...

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))



Epoch 1: val_accuracy improved from -inf to 0.65323, saving model to /kaggle/working/best_mobilenetv3_large.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 405s 886ms/step - accuracy: 0.2324 - loss: 3.5075 - val_accuracy: 0.6532 - val_loss: 1.3014 - learning_rate: 0.0010
Epoch 2/12
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 621ms/step - accuracy: 0.5667 - loss: 1.5632
Epoch 2: val_accuracy improved from 0.65323 to 0.69295, saving model to /kaggle/working/best_mobilenetv3_large.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 313s 712ms/step - accuracy: 0.5667 - loss: 1.5631 - val_accuracy: 0.6930 - val_loss: 1.1382 - learning_rate: 0.0010
Epoch 3/12
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 619ms/step - accuracy: 0.6384 - loss: 1.2824
Epoch 3: val_accuracy improved from 0.69295 to 0.71093, saving model to /kaggle/working/best_mobilenetv3_large.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 312s 709ms/step - accuracy: 0.6384 - loss: 1.2824 - val_accuracy: 0.7109 - val_loss: 1.0928 - learning_rate: 0.0010
Epoch 4/12
203/440 ━━━━━━━━━━━━━━━━━━━━ 2:25 614ms/step - accuracy: 0.6757 - loss: 1.1063